In [ ]:
from telethon.sync import TelegramClient
import re, json, os
from datetime import datetime

# 🔐 Replace with your Telegram API credentials
api_id = '******'
api_hash = '**************88'

channels = [
    'ZemenExpress', 'nevacomputer', 'meneshayeofficial', 'ethio_brand_collection',
    'Leyueqa', 'sinayelj', 'Shewabrand', 'helloomarketethiopia', 'modernshoppingcenter',
    'qnashcom', 'Fashiontera', 'kuruwear', 'gebeyaadama', 'MerttEka', 'forfreemarket',
    'classybrands', 'marakibrand', 'aradabrand2', 'marakisat2', 'belaclassic', 'AwasMart'
]

os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

In [3]:
def preprocess_amharic_text(text):
    text = text.lower()
    text = re.sub(r'[፣፡።]', ' ', text)
    text = re.sub(r'[\n\r]+', ' ', text)
    text = re.sub(r'["“”\'\'.,!?()\[\]{}<>/@#$%^&*_+=\\|-]', '', text)
    return text.strip()

def extract_entities(text):
    return {
        "price": re.findall(r'[0-9]{2,}\s?(birr|ብር)', text, re.IGNORECASE),
        "phones": re.findall(r'(\+251|0)?9[0-9]{8}', text),
        "products": re.findall(r'(ነዶ|ጫማ|ቦታ|ሸማኔ|ሱሪ|ቀሚስ|ቤት|የ[\w\s]+እቃ)', text)
    }

In [10]:

client = TelegramClient('crawler_session', api_id, api_hash)
async def crawl_and_process():
    await client.start()
    for channel in channels[:5]:  # You can change 5 to len(channels) if needed
        try:
            entity = await client.get_entity(f'https://t.me/{channel}')
            raw_messages = []
            structured = []

            async for message in client.iter_messages(entity, limit=100):
                if not message.text:
                    continue

                cleaned = preprocess_amharic_text(message.text)
                entities = extract_entities(cleaned)

                record = {
                    "channel": channel,
                    "timestamp": message.date.isoformat(),
                    "text": cleaned,
                    "raw_text": message.text,
                    "sender_id": getattr(message.sender_id, 'user_id', None),
                    "entities": entities
                }

                raw_messages.append(record)
                structured.append(record)

            # Save both raw and processed
            with open(f'../data/raw/{channel}.json', 'w', encoding='utf-8') as f:
                json.dump(raw_messages, f, ensure_ascii=False, indent=2)

            with open(f'../data/processed/{channel}_structured.json', 'w', encoding='utf-8') as f:
                json.dump(structured, f, ensure_ascii=False, indent=2)

            print(f"✅ Crawled @{channel} — {len(raw_messages)} messages")

        except Exception as e:
            print(f"❌ Error with @{channel}: {e}")